In [1]:
import gspread
import pandas as pd
from google.oauth2.service_account import Credentials

## Pipeline A

In [2]:
SERVICE_ACCOUNT_FILE = '../../key/credentials.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets',
          'https://www.googleapis.com/auth/drive']

creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
client = gspread.authorize(creds)

sheet = client.open("Chatbot Ops Report").worksheet("CSAT Chatbot")
data = sheet.get_all_values()

df = pd.DataFrame(data[1:], columns=data[0])

In [3]:
df_clean = df.copy()

df_clean = df_clean.drop('Day', axis=1)
df_clean['Date'] = pd.to_datetime(df_clean['Date'], format='%d-%b-%Y', errors='coerce')
df_clean = df_clean.dropna(subset=['Date'])

In [4]:
df_clean

,Date,Total Good Survey,Total Bad Survey,Total Responden,Total Rating,CSAT
1,2023-01-30,25,16,41,133,3.24
2,2023-01-31,26,17,43,140,3.26
3,2023-02-01,21,12,33,113,3.42
4,2023-02-02,23,17,40,130,3.25
5,2023-02-03,24,19,43,130,3.02
...,...,...,...,...,...,...
924,2025-08-10,19,9,28,100,3.57
925,2025-08-11,27,7,34,126,3.71
926,2025-08-12,20,12,32,103,3.22
927,2025-08-13,17,6,23,86,3.74


## Pipeline B

In [5]:
df_clean.to_csv('../../dataset_kula/csat.csv')